In [2]:
%matplotlib widget
import h5py
import json
import numpy as np
import multiprocessing as mp
from enum import Enum
from py4xs.detector_config import create_det_from_attrs
import matplotlib.pyplot as plt
import time
import os
from essential_func import *

os.chdir('/Users/bashit.a/Documents/Alzheimer/Dec-2020')

class h5exp():
    """ empty h5 file for exchanging exp_setup/qgrid
    """
    def __init__(self, fn, exp_setup=None):
        self.fn = fn
        if exp_setup==None:     # assume the h5 file will provide the detector config
            self.qgrid = self.read_detectors()
        else:
            self.detectors, self.qgrid = exp_setup
            self.save_detectors()
        
    def save_detectors(self):
        self.fh5 = h5py.File(self.fn, "w")   # new file
        dets_attr = [det.pack_dict() for det in self.detectors]
        self.fh5.attrs['detectors'] = json.dumps(dets_attr)
        self.fh5.attrs['qgrid'] = list(self.qgrid)
        self.fh5.flush()
        self.fh5.close()
    
    def read_detectors(self):
        self.fh5 = h5py.File(self.fn, "r")   # file must exist
        dets_attr = self.fh5.attrs['detectors']
        qgrid = self.fh5.attrs['qgrid']
        self.detectors = [create_det_from_attrs(attrs) for attrs in json.loads(dets_attr)]  
        self.fh5.close()
        return np.asarray(qgrid)

In [3]:
%matplotlib widget
de = h5exp("exp.h5")
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])
print(de.detectors[1].extension)
np.min(de.detectors[1].exp_para.Q), np.mean(de.detectors[1].exp_para.Q), np.max(de.detectors[1].exp_para.Q)


f, axs = plt.subplots(1,4, num='SAXS', figsize=(16,5))

SAXS_Q = de.detectors[0].exp_para.Q
im = axs[0].imshow(SAXS_Q, cmap="jet")
show_colorbar(im,f,axs[0])
axs[0].set_title('saxs.exp_para.Q')
print(f'exp_para.Q Min SAXS = {np.min(SAXS_Q)} \nexp_para.Q Max SAXS = {np.max(SAXS_Q)}')

SAXS_MASK = de.detectors[0].exp_para.mask.map
print('SAXS Mask Shape {}, 1.0 (white) is masked means ignore this pixel '.format(SAXS_MASK.shape))
axs[1].imshow(SAXS_MASK,cmap='gray')
axs[1].set_title('saxs.exp_para.mask.map')

im = axs[2].imshow(SAXS_Q*~SAXS_MASK, cmap='jet')
show_colorbar(im,f,axs[2])
axs[2].set_title('SAXS_Q*~SAXS_MASK')


SAXS_cor_factor = de.detectors[0].exp_para.FSA*de.detectors[0].exp_para.FPol
im = axs[3].imshow(SAXS_cor_factor, cmap='jet')
show_colorbar(im,f,axs[3])
axs[3].set_title('SAXS_cor_factor')

plt.tight_layout()

f, axs = plt.subplots(1,4, num='WAXS', figsize=(16,5))
WAXS_Q = de.detectors[1].exp_para.Q
im = axs[0].imshow(WAXS_Q, cmap="jet")
show_colorbar(im,f,axs[0])
axs[0].set_title('waxs.exp_para.Q')
print(f'exp_para.Q Min WAXS = {np.min(WAXS_Q)} \
      \nexp_para.Q Max WAXS = {np.max(WAXS_Q)}')

WAXS_MASK = de.detectors[1].exp_para.mask.map
print('WAXS Mask Shape {}, 1.0 (white) is masked means ignore this pixel '.format(WAXS_MASK.shape))
axs[1].imshow(WAXS_MASK,cmap='gray')
axs[1].set_title('waxs.exp_para.mask.map')

im = axs[2].imshow(WAXS_Q*~WAXS_MASK, cmap='jet')
show_colorbar(im,f,axs[2])
axs[2].set_title('WAXS_Q*~WAXS_MASK')

WAXS_cor_factor = de.detectors[1].exp_para.FSA*de.detectors[1].exp_para.FPol
im = axs[3].imshow(WAXS_cor_factor, cmap='jet')
show_colorbar(im,f,axs[3])
axs[3].set_title('WAXS_cor_factor')

plt.tight_layout()

_WAXS2


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

exp_para.Q Min SAXS = 0.00016180539977154286 
exp_para.Q Max SAXS = 0.37565671178795346
SAXS Mask Shape (981, 1043), 1.0 (white) is masked means ignore this pixel 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

exp_para.Q Min WAXS = 0.0008471915121232293       
exp_para.Q Max WAXS = 3.369008660772031
WAXS Mask Shape (1043, 981), 1.0 (white) is masked means ignore this pixel 


In [4]:
%matplotlib widget
WAXS_Q_mask = cv2.rectangle(WAXS_Q, (350,360), (550,660), -1, -1)
plt.imshow(WAXS_Q_mask, cmap="jet")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
de.detectors[1].ImageHeight, de.detectors[1].ImageWidth

(1043, 981)

In [5]:
print('0.001 starts at ', qgrid2[0],  'ends at ', qgrid2[44], ' 0.002 starts ', qgrid2[45], 'ends ', qgrid2[69],' 0.005 starts at ', qgrid2[70], 'ends at ', qgrid2[689])
print(WAXS_Q.min(), WAXS_Q.max())

file = '2048_B8_masked.h5'
with h5py.File(file,'r') as hdf:
    dset_waxs = np.array(hdf.get(f'{h5_top_group(file)}/primary/data/pilW2_image'))         # waxs data read from h5 file

0.001 starts at  0.005 ends at  0.048999999999999995  0.002 starts  0.05 ends  0.09800000000000005  0.005 starts at  0.1 ends at  3.195000000000003
-1.0 3.369008660772031


In [274]:
frame = 2223
dataD = dset_waxs[frame]
min_norm_scale=0.002
interpolate = True
dataD = np.ones_like(WAXS_cor_factor)
dataD = WAXS_Q_mask
maskMap = WAXS_MASK

In [310]:
def azimuthal_avg(dataD, qgrid, maskMap, cor_factor, 
            adjust_edges=True, interpolate=True, min_norm_scale=0.002):

    dd = dataD/WAXS_cor_factor                                              # divide all data by correction factor

    # Pilatus might use negative values to mark dead pixels
    idx = (dataD>=0)                                                        # idx of values >=0
    idx &= ~(maskMap)

    qd = WAXS_Q[idx].flatten()
    dd = np.asarray(dd[idx].flatten(), dtype=float)                      # 2D flattened to 1D other wise dd*dd might become negative

    qgrid = qgrid2
    adjust_edges = True

    if adjust_edges:
        # willing to throw out some data, but the edges strictly correspond to qgrid
        dq  = qgrid[1:]-qgrid[:-1]                                          # difference between two consequtive q values
        dq1 = np.hstack(([dq[0]], dq))                                      # len(dq1) == len(qgrid)

        bins = [qgrid[0]-dq1[0]/2]                                          # one value/ lower limit value qgrid[0]=0.005, dq1[0] = 0.001; bins = qgrid[0]-dq1[0]/2 = 0.0045 
        bidx = []
        binw = []

        # bins will be len(qgrid) + 1
        # bidx, binw will be len(qgrid)
        for i in range(len(qgrid)):
            el = qgrid[i] - dq1[i]/2                                        # qgrid[0]=0.005; el = qgrid[0]-dq1[0]/2 = 0.0045
            eu = qgrid[i] + dq1[i]/2                                        # qgrid[0]=0.005; eu = qgrid[0]+dq1[0]/2 = 0.0055
            if i==0 or np.fabs(el-bins[-1])<dq1[i]/100:                     # i=0 first element or 
                bins += [eu]                                                # first bin bins = [0.0045, 0.0055]
                bidx += [True]                                              # 
                binw += [dq1[i]]                                            # 
            else:
                bins += [el, eu]                                            # 
                bidx += [False, True]                                       # 
                binw += [dq1[i-1], dq1[i]]                                  # 
                #print(qgrid[i], '\n\n', bins ,'\n\n', bidx, '\n\n', binw )
                #print('.......')
    else:
        # keep all the data, but the histogrammed data will be less accurate 
        bins =  np.append([2*qgrid[0]-qgrid[1]], qgrid)
        bins += np.append(qgrid , [2*qgrid[-1]-qgrid[-2]])
        bins *= 0.5
        bidx = np.ones(len(qgrid), dtype=bool)

    norm = np.histogram(qd, bins=bins, weights=np.ones(len(qd)))[0][bidx]
    qq   = np.histogram(qd, bins=bins, weights=qd)[0][bidx]
    Iq   = np.histogram(qd, bins=bins, weights=dd)[0][bidx]
    Iq2  = np.histogram(qd, bins=bins, weights=dd*dd)[0][bidx]

    idx1 = (norm>min_norm_scale*np.arange(len(norm))**2)
    qq[idx1]  /= norm[idx1]
    Iq[idx1]  /= norm[idx1]   
    Iq2[idx1] /= norm[idx1]
    dI = np.sqrt(Iq2-Iq*Iq)
    dI[idx1] /= np.sqrt(norm[idx1])
    qq[~idx1] = np.nan
    Iq[~idx1] = np.nan
    dI[~idx1] = np.nan

    if interpolate:
        Iq = np.interp(qgrid, qq, Iq)
    return Iq, dI

In [315]:
%matplotlib widget
bins = np.array(bins)
bidx = np.array(bidx)
# print(qgrid2)
# print('........')
# print(bins)
# print('........')
data = [[0,1,1],
        [2,1,2]]
weights = [[9,10,11],
           [20,12,22]]

ones = np.ones_like(data)
hist, bin_edges = np.histogram(data, bins=[0,1,2,3], weights = ones)

hist

print(qd)
print(qd.min(), qd.max())
# print([np.round(float(i), 4) for i in bins])
plt.scatter(qgrid2,qq)

plt.figure()
plt.plot(qgrid2,norm)

f, axs = plt.subplots(1,2, num=f'frame = {frame}',figsize=(14,5))

Iq_no_Mask, _ = azimuthal_avg(WAXS_Q, qgrid2, WAXS_MASK, WAXS_cor_factor)
Iq_Masked, _ = azimuthal_avg(WAXS_Q_mask, qgrid2, WAXS_MASK, WAXS_cor_factor)

axs[0].plot(qgrid2,Iq_no_Mask,label='without')
axs[1].plot(qgrid2,Iq_Masked, label='masked')
axs[0].legend()
axs[1].legend()

len(np.where( (qd >=1.4925) & (qd <=1.4975) )[0])
#print(norm)

plt.figure()
plt.plot(np.arange(len(np.histogram(qd, bins=bins, weights=dd)[0][bidx])), np.histogram(qd, bins=bins, weights=dd)[0][bidx])


[2.90562316 2.90239235 2.89916191 ... 2.43355992 2.43590933 2.43826246]
0.28369898835062113 3.137054977863803


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-310-da9c40d8c753>:56: RuntimeWarning: invalid value encountered in sqrt
  dI = np.sqrt(Iq2-Iq*Iq)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [102]:
%matplotlib widget
idx_l, idx_u, valid_diff_values = valid_idx_search(qgrid2, Iq[np.newaxis,:])
plt.plot(qgrid2[idx_l:idx_u],Iq[idx_l:idx_u])

DD = dataD/WAXS_cor_factor
f, ax = plt.subplots()
im = plt.imshow(DD)
show_colorbar(im,f,ax)

low valid idx = 109, low valid Q = 0.295, high valid idx = 578 , high valid Q = 2.640


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
%matplotlib widget

def conv_Iq(self, qgrid, mask=None, cor_factor=1, 
            adjust_edges=True, interpolate=True, min_norm_scale=0.002):

    dd = self.data.d/cor_factor                                             # divide all data by correction factor

    # Pilatus might use negative values to mark dead pixels
    idx = (self.data.d>=0)                                                  # idx of values >=0
    if mask is not None:
        idx &= ~(mask.map)

    qd = self.exp.Q[idx].flatten()
    dd = np.asarray(dd[idx].flatten(), dtype=np.float)                      # 2D flattened to 1D other wise dd*dd might become negative

    if adjust_edges:
        # willing to throw out some data, but the edges strictly correspond to qgrid
        dq  = qgrid[1:]-qgrid[:-1]                                          # difference between two consequtive q values
        dq1 = np.hstack(([dq[0]], dq))                                      # len(dq1) == len(qgrid)

        bins = [qgrid[0]-dq1[0]/2]                                          # one value/ lower limit value qgrid[0]=0.005, dq1[0] = 0.001; bins = qgrid[0]-dq1[0]/2 = 0.0045 
        bidx = []
        binw = []

        # bins will be len(qgrid) + 1
        # bidx, binw will be len(qgrid)
        for i in range(len(qgrid)):
            el = qgrid[i] - dq1[i]/2                                        # qgrid[0]=0.005; el = qgrid[0]-dq1[0]/2 = 0.0045
            eu = qgrid[i] + dq1[i]/2                                        # qgrid[0]=0.005; eu = qgrid[0]+dq1[0]/2 = 0.0055
            if i==0 or np.fabs(el-bins[-1])<dq1[i]/100:                     # i=0 first element or 
                bins += [eu]                                                # first bin bins = [0.0045, 0.0055]
                bidx += [True]                                              # 
                binw += [dq1[i]]                                            # 
            else:
                bins += [el, eu]                                            # 
                bidx += [False, True]                                       # 
                binw += [dq1[i-1], dq1[i]]                                  # 
    else:
        # keep all the data, but the histogrammed data will be less accurate 
        bins =  np.append([2*qgrid[0]-qgrid[1]], qgrid) 
        bins += np.append(qgrid , [2*qgrid[-1]-qgrid[-2]])
        bins *= 0.5
        bidx = np.ones(len(qgrid), dtype=bool)

    norm = np.histogram(qd, bins=bins, weights=np.ones(len(qd)))[0][bidx] 
    qq   = np.histogram(qd, bins=bins, weights=qd)[0][bidx]
    Iq   = np.histogram(qd, bins=bins, weights=dd)[0][bidx]
    Iq2  = np.histogram(qd, bins=bins, weights=dd*dd)[0][bidx]

    idx1 = (norm>min_norm_scale*np.arange(len(norm))**2)
    qq[idx1] /= norm[idx1]
    Iq[idx1] /= norm[idx1]
    Iq2[idx1] /= norm[idx1]
    dI = np.sqrt(Iq2-Iq*Iq)
    dI[idx1] /= np.sqrt(norm[idx1])
    qq[~idx1] = np.nan
    Iq[~idx1] = np.nan
    dI[~idx1] = np.nan

    if interpolate:
        Iq = np.interp(qgrid, qq, Iq)

    return Iq,dI

In [3]:
class MatrixWithCoords:
    # 2D data with coordinates
    d = None
    xc = None
    yc = None 
    datatype = None
    
class DataType(Enum):
    det = 1
    qrqz = 2
    qphi = 3
    q = 4
    xyq = 5      # Cartesian version of qphi

class Data2d:
    """ 2D scattering data class
        stores the scattering pattern itself, 
    """

    def __init__(self, img, timestamp=None, uid='', exp=None, label=''):
        """ read 2D scattering pattern
            img can be either a filename (rely on Fabio to deal with the file format) or a numpy array 
        """
        self.exp = None
        self.timestamp = None
        self.uid = None
        self.data = MatrixWithCoords()
        self.qrqz_data = MatrixWithCoords()
        self.qphi_data = MatrixWithCoords()
        self.label = label
        
        if isinstance(img, np.ndarray): 
            self.im = img
            self.timestamp = timestamp
            self.uid = uid
        else:
            raise Exception('Not sure how to create Data2d from img ...')

        # self.im always stores the original image
        # self.data store the array data after the flip operation
        if exp is not None:
            self.set_exp_para(exp)
        else:
            # temporarily set data to img, without a defined exp_para
            self.flip(0)
            self.height, self.width = self.data.d.shape
            
    def set_exp_para(self, exp):
        self.flip(exp.flip)
        (self.height, self.width) = np.shape(self.data.d)
        self.exp = exp
        if exp.ImageHeight!=self.height or exp.ImageWidth!=self.width:
            raise Exception('mismatched shape between the data (%d,%d) and ExpPara (%d,%d).' % 
                            (self.width, self.height, exp.ImageWidth, exp.ImageHeight)) 
        self.data.xc = np.arange(self.width)
        self.data.yc = np.flipud(np.arange(self.height)) 
        self.data.datatype = DataType.det
        
    def flip(self, flip):
        """ this is a little complicated
            if flip<0, do a mirror operation first
            the absolute value of flip is the number of 90-deg rotations
        """  
        self.data.d = np.asarray(self.im).copy()
        if flip == 0:
            return
        if flip<0:
            self.data.d = np.fliplr(self.data.d)
            flip = -flip
        for _ in range(flip):
            self.data.d = np.rot90(self.data.d)

    
    def conv_Iq(self, qgrid, mask=None, cor_factor=1, 
                adjust_edges=True, interpolate=True, min_norm_scale=0.002):
        
        dd = self.data.d/cor_factor                                             # divide all data by correction factor
        
        # Pilatus might use negative values to mark dead pixels
        idx = (self.data.d>=0)                                                  # idx of values >=0
        if mask is not None:
            idx &= ~(mask.map)

        qd = self.exp.Q[idx].flatten()
        dd = np.asarray(dd[idx].flatten(), dtype=np.float)                      # 2D flattened to 1D other wise dd*dd might become negative

        if adjust_edges:
            # willing to throw out some data, but the edges strictly correspond to qgrid
            dq  = qgrid[1:]-qgrid[:-1]                                          # difference between two consequtive q values
            dq1 = np.hstack(([dq[0]], dq))                                      # len(dq1) == len(qgrid)

            bins = [qgrid[0]-dq1[0]/2]                                          # one value/ lower limit value qgrid[0]=0.005, dq1[0] = 0.001; bins = qgrid[0]-dq1[0]/2 = 0.0045 
            bidx = []
            binw = []
            
            # bins will be len(qgrid) + 1
            # bidx, binw will be len(qgrid)
            for i in range(len(qgrid)):
                el = qgrid[i] - dq1[i]/2                                        # qgrid[0]=0.005; el = qgrid[0]-dq1[0]/2 = 0.0045
                eu = qgrid[i] + dq1[i]/2                                        # qgrid[0]=0.005; eu = qgrid[0]+dq1[0]/2 = 0.0055
                if i==0 or np.fabs(el-bins[-1])<dq1[i]/100:                     # i=0 first element or 
                    bins += [eu]                                                # first bin bins = [0.0045, 0.0055]
                    bidx += [True]                                              # 
                    binw += [dq1[i]]                                            # 
                else:
                    bins += [el, eu]                                            # 
                    bidx += [False, True]                                       # 
                    binw += [dq1[i-1], dq1[i]]                                  # 
        else:
            # keep all the data, but the histogrammed data will be less accurate 
            bins =  np.append([2*qgrid[0]-qgrid[1]], qgrid) 
            bins += np.append(qgrid , [2*qgrid[-1]-qgrid[-2]])
            bins *= 0.5
            bidx = np.ones(len(qgrid), dtype=bool)

        norm = np.histogram(qd, bins=bins, weights=np.ones(len(qd)))[0][bidx] 
        qq   = np.histogram(qd, bins=bins, weights=qd)[0][bidx]
        Iq   = np.histogram(qd, bins=bins, weights=dd)[0][bidx]
        Iq2  = np.histogram(qd, bins=bins, weights=dd*dd)[0][bidx]

        idx1 = (norm>min_norm_scale*np.arange(len(norm))**2)
        qq[idx1] /= norm[idx1]
        Iq[idx1] /= norm[idx1]
        Iq2[idx1] /= norm[idx1]
        dI = np.sqrt(Iq2-Iq*Iq)
        dI[idx1] /= np.sqrt(norm[idx1])
        qq[~idx1] = np.nan
        Iq[~idx1] = np.nan
        dI[~idx1] = np.nan

        if interpolate:
            Iq = np.interp(qgrid, qq, Iq)

        return Iq,dI

In [4]:
font_size_list = ['xx-small', 'x-small', 'small', 'medium', 'large', 'x-large', 'xx-large']

def get_font_size(size_index):
    """ "medium" has size_index of 0
        the size_index is negative for smaller fonts and possitive for larger ones  
    """
    if size_index in font_size_list:
        i = font_size_list.index(size_index)
    else:
        i = int(size_index)+3
        if i<0:
            i = 0
        elif i>=len(font_size_list):
            i = len(font_size_list)-1
    return i-3,font_size_list[i]

class Data1d:
    def __init__(self, trandMode=None):
        self.comments = ""
        self.label = "data"
        self.overlaps = []
        self.raw_data = {}
        self.timestamp = None
        self.trans = 0
        
    def scale(self, sc):
        """
        scale the data by factor sc
        """
        if sc <= 0:
            print("scaling factor is non-positive: %f" % sc)
        self.data *= sc
        self.err *= sc
        self.trans *= sc
        self.comments += "# data is scaled by %f.\n" % sc
        if len(self.overlaps) != 0:
            for ov in self.overlaps:
                ov['raw_data1'] *= sc
                ov['raw_data2'] *= sc
                
        return self
        
    def load_from_2D(self, image, exp_para, qgrid, pre_process=None, 
                     mask=None, save_ave=False, debug=False, label=None):
        """
        image: a filename, or a Data2d instance, or a numpy array
        qgrid: for the 1D data
        exp_para: ExpPara
        mask: no longer used, extract from exp_para
        """
        self.qgrid = qgrid
        mask = exp_para.mask

        if debug==True:
            print("loading data from 2D image: ", label)
    
        if isinstance(image, Data2d):
            d2 = image
        else:
            d2 = Data2d(image, exp=exp_para)
            self.timestamp = d2.timestamp
            self.label = d2.label
            self.timestamp = d2.timestamp
            
        if label is not None:
            self.label = label
            
        # deal with things like dark current, flat field, and dezinger corrections on the 2D data
        if pre_process is not None:
            pre_process(d2.data)
        

        self.data,self.err = d2.conv_Iq(qgrid, mask,
                                        cor_factor = exp_para.FSA*exp_para.FPol)
                                        #cor_factor = exp_para.FPol)  
        if isinstance(image, np.ndarray):
            del d2      # d2 is only used temporarily
        
        if save_ave and isinstance(image, str):
            self.save(image + ".ave", debug=debug)     
        
    def plot(self, ax=None, scale=1., fontsize='large'):
        i_fs = get_font_size(fontsize)[0]
        if ax is None:
            plt.figure()
            plt.subplots_adjust(bottom=0.15)
            ax = plt.gca()
        ax.set_xlabel("$q (\AA^{-1})$", fontsize=get_font_size(i_fs)[1])
        ax.set_ylabel("$I$", fontsize=get_font_size(i_fs)[1])
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.errorbar(self.qgrid, self.data*scale, self.err*scale, label=self.label)
        for ov in self.overlaps:
            ax.plot(ov['q_overlap'], ov['raw_data1']*scale, "v")
            ax.plot(ov['q_overlap'], ov['raw_data2']*scale, "^")
        leg = ax.legend(loc='upper right', frameon=False)

        for t in leg.get_texts():
            t.set_fontsize(get_font_size(i_fs-2)[1])

In [5]:
det_names = [{"_SAXS": "pil1M_image",
              "_WAXS1": "pilW1_image",
              "_WAXS2": "pilW2_image"}, 
             {"_SAXS": "pil1M_ext_image",
              "_WAXS1": "pilW1_ext_image",
              "_WAXS2": "pilW2_ext_image"}]

class trans_mode(Enum):
    external = 0
    from_waxs = 2

def lsh5(hd, prefix='', top_only=False, silent=False):
    """ list the content of a HDF5 file
        
        hd: a handle returned by h5py.File()
        prefix: use to format the output when lsh5() is called recursively
        top_only: returns the names of the top-level groups
        silent: suppress printouts if True
    """
    if top_only:
        tp_grps = list(hd.keys())
        if not silent:
            print(tp_grps)
        return tp_grps
    for k in list(hd.keys()):
        print(prefix, k)
        if isinstance(hd[k], h5py.Group):
            print(list(hd[k].attrs.items()))
            lsh5(hd[k], prefix+"=")

def strip_name(s):
    strs = ["_SAXS","_WAXS1","_WAXS2",".cbf",".tif"]
    for ts in strs:
        if ts in s:
            ss = s.split(ts)
            s = "".join(ss)
    return s
            
def common_name(s1, s2):
    s1 = strip_name(s1)
    s2 = strip_name(s2)
    l = len(s1)
    if len(s2) < l:
        l = len(s2)

    s = ""
    for i in range(l):
        if s1[i] == s2[i]:
            s += s1[i]
        else:
            break
    if len(s) < 1:
        s = s1.copy()
    return s.rstrip("-_ ")
            
def merge_d1s(d1s, detectors, save_merged=False, debug=False):
    """ utility function to merge 1D data sets, using functions under slnxs 
        d1s should contain data corresponding to detectors
    """
    s0 = Data1d()
    s0.qgrid = d1s[0].qgrid
    d_tot = np.zeros(s0.qgrid.shape)
    d_max = np.zeros(s0.qgrid.shape)
    d_min = np.zeros(s0.qgrid.shape)+1.e32
    e_tot = np.zeros(s0.qgrid.shape)
    c_tot = np.zeros(s0.qgrid.shape)
    label = None
    comments = ""
                
    for d1 in d1s:        
        # empty part of the data is nan
        idx = ~np.isnan(d1.data)
        d_tot[idx] += d1.data[idx]
        e_tot[idx] += d1.err[idx]
        c_tot[idx] += 1

        idx1 = (np.ma.fix_invalid(d1.data, fill_value=-1)>d_max).data
        d_max[idx1] = d1.data[idx1]
        idx2 = (np.ma.fix_invalid(d1.data, fill_value=1e32)<d_min).data
        d_min[idx2] = d1.data[idx2]
            
        comments += d1.comments
        if label is None:
            label = d1.label
        else:
            label = common_name(label, d1.label)
        
    s0.data = d_tot
    s0.err = e_tot
    idx = (c_tot>1)
    s0.overlaps.append({'q_overlap': s0.qgrid[idx],
                        'raw_data1': d_max[idx],
                        'raw_data2': d_min[idx]})
    s0.data[idx] /= c_tot[idx]
    s0.err[idx] /= np.sqrt(c_tot[idx])
    s0.label = label
    s0.comments = comments # .replace("# ", "## ")
    if save_merged:
        s0.save(s0.label+".dd", debug=debug)
        
    return s0
            
            
def proc_d1merge(args):
    """ utility function to perfrom azimuthal average and merge detectors
    """
    images,sn,nframes,starting_frame_no,debug,detectors,qgrid,reft,save_1d,save_merged = args
    ret = {'merged': []}
    sc = {}
    
    for det in detectors:
        ret[det.extension] = []
        if det.fix_scale is not None:
            sc[det.extension] = 1./det.fix_scale

    if debug is True:
        print("processing started: sample = %s, starting frame = #%d" % (sn, starting_frame_no))
    for i in range(nframes):
        for det in detectors:
            dt = Data1d()
            label = "%s_f%05d%s" % (sn, i+starting_frame_no, det.extension)
            dt.load_from_2D(images[det.extension][i], 
                            det.exp_para, qgrid, det.pre_process, det.exp_para.mask,
                            save_ave=False, debug=debug, label=label)
            dt.scale(sc[det.extension])
            ret[det.extension].append(dt)
    
        dm = merge_d1s([ret[det.extension][i] for det in detectors], detectors, save_merged, debug)
        ret['merged'].append(dm)
            
    if debug is True:
        print("processing completed: ", sn, starting_frame_no)

    return [sn, starting_frame_no, ret]            

def pack_d1(data, ret_trans=True):
    """ utility function to creat a list of [intensity, error] from a Data1d object 
        or from a list of Data1s objects
    """
    if isinstance(data, Data1d):
        if ret_trans:
            return np.asarray([data.data,data.err]), data.trans
        else:
            return np.asarray([data.data,data.err])
    elif isinstance(data, list):
        tvs = [d.trans for d in data]
        return np.asarray([pack_d1(d, False) for d in data]),tvs

In [6]:
class h5xs():
    """ Scattering data in transmission geometry
        Transmitted beam intensity can be set either from the water peak (sol), or from intensity monitor.
        Data processing can be done either in series, or in parallel. Serial processing can be forced.
        
    """    
    def __init__(self, fn, exp_setup=None, transField='', save_d1=True):
        """ exp_setup: [detectors, qgrid]
            transField: the intensity monitor field packed by suitcase from databroker
            save_d1: save newly processed 1d data back to the h5 file
        """
        self.d1s = {}
        self.detectors = None
        self.samples = []
        self.attrs = {}
        # name of the dataset that contains transmitted beam intensity, e.g. em2_current1_mean_value
        self.transField = None  

        self.fn = fn
        self.save_d1 = save_d1
        self.fh5 = h5py.File(self.fn, "r+")   # file must exist
        if exp_setup==None:     # assume the h5 file will provide the detector config
            self.qgrid = self.read_detectors()
        else:
            self.detectors, self.qgrid = exp_setup
            self.save_detectors()
        self.list_samples(quiet=True)
        # find out what are the fields corresponding to the 2D detectors
        # at LiX there are two possibilities
        data_fields = list(self.fh5[self.samples[0]+'/primary/data'])
        self.det_name = None
        # these are the detectors that are present in the data
        d_dn = [d.extension for d in self.detectors]
        for det_name in det_names:
            for k in set(det_name.keys()).difference(d_dn):
                del det_name[k]
            if set(det_name.values()).issubset(data_fields):
                self.det_name = det_name
                break
        if self.det_name is None:
            print('fields in the h5 file: ', data_fields)
            raise Exception("Could not find the data corresponding to the detectors.")
        if transField=='':
            # "2," --> self.transField = '' and self.transMode = trans_mode.from_waxs
            if 'trans' in self.fh5.attrs:
                [v, self.transField] = self.fh5.attrs['trans'].split(',')
                self.transMode = trans_mode(int(v))
                return
            else:
                self.transMode = trans_mode.from_waxs
                self.transField = ''
        elif transField not in data_fields:
            print("invalid filed for transmitted intensity: ", transField)
            raise Exception()
        else:
            self.transField = transField
            self.transMode = trans_mode.external
        self.fh5.attrs['trans'] = ','.join([str(self.transMode.value), self.transField])  # "0,em2_sum_all_mean_value"
        self.fh5.flush()
            
    def save_detectors(self):
        dets_attr = [det.pack_dict() for det in self.detectors]
        self.fh5.attrs['detectors'] = json.dumps(dets_attr)
        self.fh5.attrs['qgrid'] = list(self.qgrid)
        self.fh5.flush()
    
    def read_detectors(self):
        dets_attr = self.fh5.attrs['detectors']
        qgrid = self.fh5.attrs['qgrid']
        self.detectors = [create_det_from_attrs(attrs) for attrs in json.loads(dets_attr)]  
        return np.asarray(qgrid)
    
    def list_samples(self, quiet=False):
        self.samples = lsh5(self.fh5, top_only=True, silent=True)
        if not quiet:
            print(self.samples)
    
    def save_d1s(self, sn=None, debug=False):
        """
        save the 1d data in memory to the hdf5 file 
        processed data go under the group sample_name/processed
        assume that the shape of the data is unchanged
        """
        
        if self.save_d1 is False:
            print("requested to save_d1s() but h5xs.save_d1 is False.")
            return
        if sn==None:
            self.list_samples(quiet=True)
            for sn in self.samples:
                self.save_d1s(sn)
        
        fh5 = self.fh5        
        if "processed" not in list(lsh5(fh5[sn], top_only=True, silent=True)):
            grp = fh5[sn].create_group("processed")
        else:
            grp = fh5[sn+'/processed']
            g0 = lsh5(grp, top_only=True, silent=True)[0]
            if grp[g0][0].shape[1]!=len(self.qgrid): # if grp[g0].value[0].shape[1]!=len(self.qgrid):
                # new size for the data
                del fh5[sn+'/processed']
                grp = fh5[sn].create_group("processed")
        
        # these attributes are not necessarily available when save_d1s() is called
        if sn in list(self.attrs.keys()):
            for k in list(self.attrs[sn].keys()):
                grp.attrs[k] = self.attrs[sn][k]
                if debug is True:
                    print("writting attribute to %s: %s" % (sn, k))

        ds_names = lsh5(grp, top_only=True, silent=True)
        for k in list(self.d1s[sn].keys()):
            data,tvs = pack_d1(self.d1s[sn][k])
            if debug is True:
                print("writting attribute to %s: %s" % (sn, k))
            if k not in ds_names:
                grp.create_dataset(k, data=data)
            else:
                grp[k][...] = data   

            # save trans values for processed data
            # before 1d data merge, the trans value should be 0              
            # on the other hand there could be data collected with the beam off, therefore trans=0
            if (np.asarray(tvs)>0).any(): 
                grp[k].attrs['trans'] = tvs
                
        fh5.flush()
            
    def load_data(self, update_only=False, detectors=None,
           reft=-1, save_1d=False, save_merged=False, debug=False, N=8, max_c_size=0):
        """ assume multiple samples, parallel-process by sample
            use Pool to limit the number of processes; 
            access h5 group directly in the worker process
        """
        if debug is True:
            print("start processing: load_data()")
            t1 = time.time()
        
        fh5 = self.fh5
        self.samples = lsh5(fh5, top_only=True, silent=(not debug))
        
        results = {}
        pool = mp.Pool(N)
        jobs = []
        
        for sn in self.samples:
            if sn not in list(self.attrs.keys()):
                self.attrs[sn] = {}
            if 'buffer' in list(fh5[sn].attrs):
                self.buffer_list[sn] = fh5[sn].attrs['buffer'].split('  ')
            if update_only and sn in list(self.d1s.keys()):
                self.load_d1s(sn)   # load processed data saved in the file
                continue
                                    
            self.d1s[sn] = {}
            results[sn] = {}
            dset = fh5["%s/primary/data" % sn]
            
            s = dset["%s" % self.det_name[self.detectors[0].extension]].shape
            if len(s)==3 or len(s)==4:
                self.n_total_frames = s[0]
            else:
                raise Exception("don't know how to handle shape:", )
            if self.n_total_frames<N*N/2:
                Np = 1
                c_size = N
            else:
                Np = N
                c_size = int(self.n_total_frames/N)
                if max_c_size>0 and c_size>max_c_size:
                    Np = int(self.n_total_frames/max_c_size)+1
                    c_size = int(self.n_total_frames/Np)
                    
            # process data in group in hope to limit memory use
            # the raw data could be stored in a 1d or 2d array
            if detectors is None:
                detectors = self.detectors
            for i in range(Np):
                if i==Np-1:
                    nframes = self.n_total_frames - c_size*(Np-1)   # 3721 - 465*(7) = 466
                else:
                    nframes = c_size    # 465
                    
                if len(s)==3:
                    images = {}
                    for det in detectors:
                        gn = f'{self.det_name[det.extension]}'
                        images[det.extension] = dset[gn][i*c_size:i*c_size+nframes]    

                    if N>1: # multi-processing, need to keep track of total number of active processes                    
                        job = pool.map_async(proc_d1merge, [(images, sn, nframes, i*c_size, debug,
                                                             detectors, self.qgrid, reft, save_1d, save_merged)])
                        jobs.append(job)
                    else: # serial processing
                        [sn, fr1, data] = proc_d1merge((images, sn, nframes, i*c_size, debug, 
                                                        detectors, self.qgrid, reft, save_1d, save_merged)) 
                        results[sn][fr1] = data                
                else: # len(s)==4
                    for j in range(s[1]):
                        images = {}
                        for det in detectors:
                            gn = f'{self.det_name[det.extension]}'
                            images[det.extension] = dset[gn][i*c_size:i*c_size+nframes, j]
                        if N>1: # multi-processing, need to keep track of total number of active processes
                            job = pool.map_async(proc_d1merge, [(images, sn, nframes, i*c_size+j*s[0], debug,
                                                                 detectors, self.qgrid, reft, save_1d, save_merged)])
                            jobs.append(job)
                        else: # serial processing
                            [sn, fr1, data] = proc_d1merge((images, sn, nframes, i*c_size+j*s[0], debug, 
                                                            detectors, self.qgrid, reft, save_1d, save_merged)) 
                            results[sn][fr1] = data                

        if N>1:
            for job in jobs:
                [sn, fr1, data] = job.get()[0]
                results[sn][fr1] = data           # [sn, starting_frame_no, ret]
                print("data received: sn=%s, fr1=%d" % (sn,fr1) )
            pool.close()
            pool.join()

        for sn in self.samples:
            if sn not in results.keys():
                continue
            data = {}
            frns = list(results[sn].keys())
            frns.sort()
            for k in results[sn][frns[0]].keys():
                data[k] = []  # ret = {'merged': []}
                for frn in frns:                             # dm = merge_d1s([ret[det.extension][i] for det in detectors], detectors, save_merged, debug)
                    data[k].extend(results[sn][frn][k])      # ret['merged'].append(dm)
            self.d1s[sn] = data           
        
        self.save_d1s(debug=debug)
        if debug is True:
            t2 = time.time()
            print("done, time lapsed: %.2f sec" % (t2-t1))

In [7]:
if __name__ == '__main__':
    dt  = h5xs("mica.h5", [de.detectors, qgrid2])
    dt.load_data(N=4, debug=True)

ValueError: too many values to unpack (expected 2)

In [ ]:
print(dt.n_total_frames)
n = len(np.asarray(dt.d1s['2048_B8']['_WAXS2'][0].data))
diff_patterns = np.zeros((1,n))
for i in range(dt.n_total_frames):
    diff_patterns = np.vstack([diff_patterns, np.asarray(dt.d1s['1934_B8']['merged'][i].data).reshape(1,n)])

diff_patterns = np.delete(diff_patterns,obj=0,axis=0)
diff_patterns = np.expand_dims(diff_patterns,axis=1)
diff_patterns = np.transpose(diff_patterns,axes=[1,2,0])        # reshape (1, 690, 3721)
print(diff_patterns.shape)

BandAngles = qgrid2
print(f'BandAngles shape = {BandAngles.shape}')                 # shape (690,)

In [6]:
import numpy as np
import scipy.io as sio
from matplotlib import pyplot  as plt
import seaborn as sns

[_,n_intensity,n_patterns] = np.shape(diff_patterns);   
Width = int(np.sqrt(n_patterns));
Height = int(np.sqrt(n_patterns));    
print('Intensity ' + str(n_intensity) + ' Diff. Patterns ' + str(Width))


# -*- Find Maximum Value -*- 
_, QMAX, SMAX = np.where(diff_patterns == np.max(diff_patterns))    # Maximum Q and Maximum Diff. Patt.

# -*- Plot Intensity -*- 
PlotDiffRange = np.linspace(SMAX,SMAX + 19, 20,dtype=np.int32);    # 20 Plots from Highest Intensity 
l=0;

plt.figure;
fig,axes = plt.subplots(5,4, figsize=(15,8), sharex=True, sharey=True);
R,C = np.shape(axes)
for r in range(R):
    for c in range(C):
        axes[r][c].plot(BandAngles, np.log(np.ravel(diff_patterns[:,:,PlotDiffRange[l]])))
        axes[r][c].set_title('Sample = ' + str(PlotDiffRange[l] + 1), fontsize=8)
        l = l + 1;
plt.setp(axes[-1,:], xlabel = 'Q')
plt.setp(axes[:,0],  ylabel = 'log Intensity')
fig.show()

# -*- Plot Heat Map -*- 
plt.figure(figsize=(16,9))
circ_value = np.zeros([1,1,n_patterns]);             # circ_avg. value
for l in range(n_intensity) :
    circ_value = circ_value + diff_patterns[:,l,:];
circ_value = circ_value/n_intensity;
img_orig = np.reshape(circ_value,[Height,Width]);
ll = sns.heatmap(img_orig)
fig.show()

NameError: name 'diff_patterns' is not defined

In [7]:
print('SAXS [{} , {}] WAXS [{} , {}] '.format(qgrid2[2], qgrid2[124], qgrid2[109], qgrid2[578]) )

SAXS [0.007 , 0.3700000000000002] WAXS [0.29500000000000015 , 2.6400000000000023] 


In [92]:
print('SAXS Fix scale {}. \nWAXS Fix scale {}'.format(1/de.detectors[0].fix_scale, \
                                                    1/de.detectors[1].fix_scale))

SAXS Fix scale 1.0. 
WAXS Fix scale 0.010404022787155612
